 # 05 - Predictive Maintenance Binary Classification
 Predictive maintenance techniques are designed to help determine the condition of in-service equipment in order to
 estimate when maintenance should be performed. Predictive maintenance can be modeled in several ways,
 1. Predict the Remaining Useful Life (RUL), or Time to Failure (TTF)
 2. Predict if the asset will fail by given a certain time frame
 3. Predict critical level of the asset by give a certain time frame

 This example we will look at the 2nd modeling strategy which is to predict weather the asset is going to fail.
 The label consist of 0 and 1. 0 means the assets is working fine and 1 means it require maintenance.

In [2]:
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sys
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torch
from torch import nn, optim
import torch.nn.functional as F

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# reading dataset
df_train = pd.read_csv("../datasets/predictive_maintenance/train.csv")
df_test = pd.read_csv("../datasets/predictive_maintenance/test.csv")

Data Analysis

In [4]:
df_train.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,cycle_norm,RUL,label1,label2
0,1,1,0.459770,0.166667,0,0,0.183735,0.406802,0.309757,0,...,0,0.333333,0,0,0.713178,0.724662,0.00000,191,0,0
1,1,2,0.609195,0.250000,0,0,0.283133,0.453019,0.352633,0,...,0,0.333333,0,0,0.666667,0.731014,0.00277,190,0,0
2,1,3,0.252874,0.750000,0,0,0.343373,0.369523,0.370527,0,...,0,0.166667,0,0,0.627907,0.621375,0.00554,189,0,0
3,1,4,0.540230,0.500000,0,0,0.343373,0.256159,0.331195,0,...,0,0.333333,0,0,0.573643,0.662386,0.00831,188,0,0
4,1,5,0.390805,0.333333,0,0,0.349398,0.257467,0.404625,0,...,0,0.416667,0,0,0.589147,0.704502,0.01108,187,0,0


In [5]:
print('Classes in train dataset:', Counter(df_train["label1"])) # show the class distribution, and here we observe
                                                                # high imbalance with ratio of around 3.5 : 1
df_train.isna().sum() # shows that there are no missing values

Classes in train dataset: Counter({0: 17531, 1: 3100})


id            0
cycle         0
setting1      0
setting2      0
setting3      0
s1            0
s2            0
s3            0
s4            0
s5            0
s6            0
s7            0
s8            0
s9            0
s10           0
s11           0
s12           0
s13           0
s14           0
s15           0
s16           0
s17           0
s18           0
s19           0
s20           0
s21           0
cycle_norm    0
RUL           0
label1        0
label2        0
dtype: int64

In [6]:
# let's first remove the unnecessary columns
drop_columns_list = ["setting3", "s1", "s5", "s10", "s16", "s18", "s19", "RUL", "label2"]
df_train = df_train.drop(drop_columns_list, axis=1)
df_test = df_test.drop(drop_columns_list, axis=1)

In [7]:
df_train.head()

,id,cycle,setting1,setting2,s2,s3,s4,s6,s7,s8,...,s11,s12,s13,s14,s15,s17,s20,s21,cycle_norm,label1
0,1,1,0.459770,0.166667,0.183735,0.406802,0.309757,1,0.726248,0.242424,...,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662,0.00000,0
1,1,2,0.609195,0.250000,0.283133,0.453019,0.352633,1,0.628019,0.212121,...,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014,0.00277,0
2,1,3,0.252874,0.750000,0.343373,0.369523,0.370527,1,0.710145,0.272727,...,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375,0.00554,0
3,1,4,0.540230,0.500000,0.343373,0.256159,0.331195,1,0.740741,0.318182,...,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386,0.00831,0
4,1,5,0.390805,0.333333,0.349398,0.257467,0.404625,1,0.668277,0.242424,...,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502,0.01108,0


In [8]:
# let's also check for the data types
df_train.dtypes

id              int64
cycle           int64
setting1      float64
setting2      float64
s2            float64
s3            float64
s4            float64
s6              int64
s7            float64
s8            float64
s9            float64
s11           float64
s12           float64
s13           float64
s14           float64
s15           float64
s17           float64
s20           float64
s21           float64
cycle_norm    float64
label1          int64
dtype: object

In [9]:
y_train = df_train['label1'].to_numpy() # convert dtype to numpy so that can be easily converted to torch tensor
x_train = df_train.drop('label1', axis=1)
y_test = df_test['label1'].to_numpy()
x_test = df_test.drop('label1', axis=1)

print('Shape for y of train dataset: ', y_train.shape)
print('Shape for x of train dataset: ', x_train.shape)
print('Shape for y of test dataset: ', y_test.shape)
print('Shape for x of test dataset: ', x_test.shape)



# data pre-processing : min-max normalization
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# # train_ds = TensorDataset(torch.Tensor(np.array(train_samples)), torch.Tensor(np.array(train_labels)))
# train_ds = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
# test_ds = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
# print(train_ds[:5])

Shape for y of train dataset:  (20631,)
Shape for x of train dataset:  (20631, 20)
Shape for y of test dataset:  (13096,)
Shape for x of test dataset:  (13096, 20)


In [10]:
# Let's also do a one-hot encoding for our label first since we plan to use nn.BCEwithLogitsLoss() as criterion (loss function)
y_train = torch.nn.functional.one_hot(torch.Tensor(y_train).to(torch.int64))
y_test = torch.nn.functional.one_hot(torch.Tensor(y_test).to(torch.int64))

# So by right, the dtype now should be Torch tensor and has 2 columns (since 2 classes)
print(y_train.shape)
print(y_test.shape)

torch.Size([20631, 2])
torch.Size([13096, 2])


In [11]:
def data_processor(x_data, y_data, sequence_length):
    """
    Helper function to sample sub-sequence of training data.
    Input data must be numpy.
    """
    x, y = [], []

    # Fill the batch with random sequences of data
    for i in range(x_data.shape[0] - sequence_length):

        # copy the sequences of data starting at this index
        x.append(x_data[i:i + sequence_length])
        y.append(y_data[i + sequence_length])
    
    return x, y

In [12]:
# let's change the data to a suitable format for model to train on
x_sequence_train, y_sequence_train = data_processor(x_train, y_train, 30)
x_sequence_test, y_sequence_test = data_processor(x_test, y_test, 30)

# let's do a sanity check too
print("Total samples for X train: " + str(len(x_sequence_train)))
print("Total samples for y train: " + str(len(y_sequence_train)))
print("Total samples for X test: " + str(len(x_sequence_test)))
print("Total samples for y test: " + str(len(y_sequence_test)))

Total samples for X train: 20601
Total samples for y train: 20601
Total samples for X test: 13066
Total samples for y test: 13066


In [13]:
# Let us use the Dataset object to instantiate our dataset, this way it enables the use of len and indexing
# This is the preferred way of preparing data in Pytorch
class TensorDataset(torch.utils.data.dataset.Dataset):
    def __init__(self, x, y):
        self.x = torch.Tensor(x)
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [14]:
# creating train and test datasets
train_ds = TensorDataset(x_sequence_train, y_sequence_train)
test_ds = TensorDataset(x_sequence_test, y_sequence_test)

# let us print out the first two rows of train dataset and check the shape of dataset too
print(train_ds[:2])
print(len(train_ds))          # print out number of samples
print(len(train_ds[0]))       # print out ?
print(len(train_ds[0][0]))    # print out number of sequences / sequence length
print(len(train_ds[0][0][0])) # print out number of features 

(tensor([[[0.0000, 0.0000, 0.4598,  ..., 0.7132, 0.7247, 0.0000],
         [0.0000, 0.0028, 0.6092,  ..., 0.6667, 0.7310, 0.0028],
         [0.0000, 0.0055, 0.2529,  ..., 0.6279, 0.6214, 0.0055],
         ...,
         [0.0000, 0.0748, 0.3621,  ..., 0.6744, 0.5384, 0.0748],
         [0.0000, 0.0776, 0.5690,  ..., 0.6124, 0.6428, 0.0776],
         [0.0000, 0.0803, 0.3736,  ..., 0.7054, 0.7136, 0.0803]],

        [[0.0000, 0.0028, 0.6092,  ..., 0.6667, 0.7310, 0.0028],
         [0.0000, 0.0055, 0.2529,  ..., 0.6279, 0.6214, 0.0055],
         [0.0000, 0.0083, 0.5402,  ..., 0.5736, 0.6624, 0.0083],
         ...,
         [0.0000, 0.0776, 0.5690,  ..., 0.6124, 0.6428, 0.0776],
         [0.0000, 0.0803, 0.3736,  ..., 0.7054, 0.7136, 0.0803],
         [0.0000, 0.0831, 0.5805,  ..., 0.6202, 0.6091, 0.0831]]]), [tensor([1, 0]), tensor([1, 0])])
20601
2
30
20


In [15]:
print(len(test_ds))
print(len(test_ds))          # print out number of samples
print(len(test_ds[0]))       # print out ?
print(len(test_ds[0][0]))    # print out number of sequences / sequence length
print(len(test_ds[0][0][0])) # print out number of label columns 

13066
13066
2
30
20


In [16]:
print(train_ds[0][1])

tensor([1, 0])


In [17]:
# Now, we are ready to create iterator using DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_ds,
                                          batch_size=200,
                                          shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_ds,
                                          batch_size=1,
                                          shuffle=False)

In [18]:
# Or we can also just load one batch of the iterator as checking
for i, (x, y) in enumerate(train_loader):
    print(x)

tensor([[[0.0000, 0.0000, 0.4598,  ..., 0.7132, 0.7247, 0.0000],
         [0.0000, 0.0028, 0.6092,  ..., 0.6667, 0.7310, 0.0028],
         [0.0000, 0.0055, 0.2529,  ..., 0.6279, 0.6214, 0.0055],
         ...,
         [0.0000, 0.0748, 0.3621,  ..., 0.6744, 0.5384, 0.0748],
         [0.0000, 0.0776, 0.5690,  ..., 0.6124, 0.6428, 0.0776],
         [0.0000, 0.0803, 0.3736,  ..., 0.7054, 0.7136, 0.0803]],

        [[0.0000, 0.0028, 0.6092,  ..., 0.6667, 0.7310, 0.0028],
         [0.0000, 0.0055, 0.2529,  ..., 0.6279, 0.6214, 0.0055],
         [0.0000, 0.0083, 0.5402,  ..., 0.5736, 0.6624, 0.0083],
         ...,
         [0.0000, 0.0776, 0.5690,  ..., 0.6124, 0.6428, 0.0776],
         [0.0000, 0.0803, 0.3736,  ..., 0.7054, 0.7136, 0.0803],
         [0.0000, 0.0831, 0.5805,  ..., 0.6202, 0.6091, 0.0831]],

        [[0.0000, 0.0055, 0.2529,  ..., 0.6279, 0.6214, 0.0055],
         [0.0000, 0.0083, 0.5402,  ..., 0.5736, 0.6624, 0.0083],
         [0.0000, 0.0111, 0.3908,  ..., 0.5891, 0.7045, 0.

tensor([[[0.0505, 0.2327, 0.5230,  ..., 0.5504, 0.3502, 0.2327],
         [0.0505, 0.2355, 0.5172,  ..., 0.3953, 0.5358, 0.2355],
         [0.0505, 0.2382, 0.2011,  ..., 0.3953, 0.5489, 0.2382],
         ...,
         [0.0505, 0.3075, 0.3678,  ..., 0.4341, 0.5787, 0.3075],
         [0.0505, 0.3102, 0.3161,  ..., 0.3721, 0.5109, 0.3102],
         [0.0505, 0.3130, 0.4368,  ..., 0.4186, 0.4412, 0.3130]],

        [[0.0505, 0.2355, 0.5172,  ..., 0.3953, 0.5358, 0.2355],
         [0.0505, 0.2382, 0.2011,  ..., 0.3953, 0.5489, 0.2382],
         [0.0505, 0.2410, 0.4540,  ..., 0.5969, 0.5954, 0.2410],
         ...,
         [0.0505, 0.3102, 0.3161,  ..., 0.3721, 0.5109, 0.3102],
         [0.0505, 0.3130, 0.4368,  ..., 0.4186, 0.4412, 0.3130],
         [0.0505, 0.3158, 0.4253,  ..., 0.4729, 0.5276, 0.3158]],

        [[0.0505, 0.2382, 0.2011,  ..., 0.3953, 0.5489, 0.2382],
         [0.0505, 0.2410, 0.4540,  ..., 0.5969, 0.5954, 0.2410],
         [0.0505, 0.2438, 0.4655,  ..., 0.3798, 0.4778, 0.

         [0.0909, 0.3158, 0.3333,  ..., 0.6822, 0.6060, 0.3158]]])
tensor([[[0.0909, 0.2382, 0.5115,  ..., 0.7287, 0.7180, 0.2382],
         [0.0909, 0.2410, 0.3276,  ..., 0.7442, 0.7509, 0.2410],
         [0.0909, 0.2438, 0.5057,  ..., 0.6434, 0.6816, 0.2438],
         ...,
         [0.0909, 0.3130, 0.4023,  ..., 0.7597, 0.7405, 0.3130],
         [0.0909, 0.3158, 0.3333,  ..., 0.6822, 0.6060, 0.3158],
         [0.0909, 0.3186, 0.2414,  ..., 0.6744, 0.7410, 0.3186]],

        [[0.0909, 0.2410, 0.3276,  ..., 0.7442, 0.7509, 0.2410],
         [0.0909, 0.2438, 0.5057,  ..., 0.6434, 0.6816, 0.2438],
         [0.0909, 0.2465, 0.4368,  ..., 0.7829, 0.6930, 0.2465],
         ...,
         [0.0909, 0.3158, 0.3333,  ..., 0.6822, 0.6060, 0.3158],
         [0.0909, 0.3186, 0.2414,  ..., 0.6744, 0.7410, 0.3186],
         [0.0909, 0.3213, 0.4713,  ..., 0.6822, 0.7005, 0.3213]],

        [[0.0909, 0.2438, 0.5057,  ..., 0.6434, 0.6816, 0.2438],
         [0.0909, 0.2465, 0.4368,  ..., 0.7829, 0.6930, 

tensor([[[0.1616, 0.2632, 0.5575,  ..., 0.6744, 0.7428, 0.2632],
         [0.1616, 0.2659, 0.3506,  ..., 0.5969, 0.7901, 0.2659],
         [0.1616, 0.2687, 0.4195,  ..., 0.6744, 0.6194, 0.2687],
         ...,
         [0.1616, 0.3380, 0.4655,  ..., 0.6202, 0.5301, 0.3380],
         [0.1616, 0.3407, 0.3448,  ..., 0.7442, 0.5664, 0.3407],
         [0.1616, 0.3435, 0.4195,  ..., 0.6512, 0.6808, 0.3435]],

        [[0.1616, 0.2659, 0.3506,  ..., 0.5969, 0.7901, 0.2659],
         [0.1616, 0.2687, 0.4195,  ..., 0.6744, 0.6194, 0.2687],
         [0.1616, 0.2715, 0.4943,  ..., 0.5814, 0.6946, 0.2715],
         ...,
         [0.1616, 0.3407, 0.3448,  ..., 0.7442, 0.5664, 0.3407],
         [0.1616, 0.3435, 0.4195,  ..., 0.6512, 0.6808, 0.3435],
         [0.1616, 0.3463, 0.4598,  ..., 0.5736, 0.6432, 0.3463]],

        [[0.1616, 0.2687, 0.4195,  ..., 0.6744, 0.6194, 0.2687],
         [0.1616, 0.2715, 0.4943,  ..., 0.5814, 0.6946, 0.2715],
         [0.1616, 0.2742, 0.0862,  ..., 0.7209, 0.6520, 0.

         [0.2121, 0.1801, 0.4885,  ..., 0.4264, 0.4917, 0.1801]]])
tensor([[[0.2121, 0.1025, 0.4655,  ..., 0.6357, 0.5233, 0.1025],
         [0.2121, 0.1053, 0.2701,  ..., 0.4109, 0.4482, 0.1053],
         [0.2121, 0.1080, 0.6839,  ..., 0.5581, 0.4856, 0.1080],
         ...,
         [0.2121, 0.1773, 0.5977,  ..., 0.4961, 0.6087, 0.1773],
         [0.2121, 0.1801, 0.4885,  ..., 0.4264, 0.4917, 0.1801],
         [0.2121, 0.1828, 0.4540,  ..., 0.3643, 0.5367, 0.1828]],

        [[0.2121, 0.1053, 0.2701,  ..., 0.4109, 0.4482, 0.1053],
         [0.2121, 0.1080, 0.6839,  ..., 0.5581, 0.4856, 0.1080],
         [0.2121, 0.1108, 0.4885,  ..., 0.5426, 0.4141, 0.1108],
         ...,
         [0.2121, 0.1801, 0.4885,  ..., 0.4264, 0.4917, 0.1801],
         [0.2121, 0.1828, 0.4540,  ..., 0.3643, 0.5367, 0.1828],
         [0.2121, 0.1856, 0.6264,  ..., 0.4884, 0.6599, 0.1856]],

        [[0.2121, 0.1080, 0.6839,  ..., 0.5581, 0.4856, 0.1080],
         [0.2121, 0.1108, 0.4885,  ..., 0.5426, 0.4141, 

...

         [0.8283, 0.0997, 0.3793,  ..., 0.7287, 0.8155, 0.0997]]])
tensor([[[0.8283, 0.0222, 0.5345,  ..., 0.5814, 0.7673, 0.0222],
         [0.8283, 0.0249, 0.5402,  ..., 0.7132, 0.7771, 0.0249],
         [0.8283, 0.0277, 0.3793,  ..., 0.6667, 0.6774, 0.0277],
         ...,
         [0.8283, 0.0970, 0.7241,  ..., 0.7364, 0.7479, 0.0970],
         [0.8283, 0.0997, 0.3793,  ..., 0.7287, 0.8155, 0.0997],
         [0.8283, 0.1025, 0.5805,  ..., 0.5426, 0.7404, 0.1025]],

        [[0.8283, 0.0249, 0.5402,  ..., 0.7132, 0.7771, 0.0249],
         [0.8283, 0.0277, 0.3793,  ..., 0.6667, 0.6774, 0.0277],
         [0.8283, 0.0305, 0.5920,  ..., 0.5969, 0.7668, 0.0305],
         ...,
         [0.8283, 0.0997, 0.3793,  ..., 0.7287, 0.8155, 0.0997],
         [0.8283, 0.1025, 0.5805,  ..., 0.5426, 0.7404, 0.1025],
         [0.8283, 0.1053, 0.4253,  ..., 0.6512, 0.6442, 0.1053]],

        [[0.8283, 0.0277, 0.3793,  ..., 0.6667, 0.6774, 0.0277],
         [0.8283, 0.0305, 0.5920,  ..., 0.5969, 0.7668, 

         [0.8586, 0.2438, 0.4483,  ..., 0.6202, 0.6397, 0.2438]]])
tensor([[[0.8586, 0.1662, 0.5805,  ..., 0.4186, 0.5264, 0.1662],
         [0.8586, 0.1690, 0.5805,  ..., 0.5969, 0.6658, 0.1690],
         [0.8586, 0.1717, 0.4713,  ..., 0.5814, 0.5380, 0.1717],
         ...,
         [0.8586, 0.2410, 0.4540,  ..., 0.4961, 0.6263, 0.2410],
         [0.8586, 0.2438, 0.4483,  ..., 0.6202, 0.6397, 0.2438],
         [0.8586, 0.2465, 0.4943,  ..., 0.5116, 0.5539, 0.2465]],

        [[0.8586, 0.1690, 0.5805,  ..., 0.5969, 0.6658, 0.1690],
         [0.8586, 0.1717, 0.4713,  ..., 0.5814, 0.5380, 0.1717],
         [0.8586, 0.1745, 0.4310,  ..., 0.7054, 0.7487, 0.1745],
         ...,
         [0.8586, 0.2438, 0.4483,  ..., 0.6202, 0.6397, 0.2438],
         [0.8586, 0.2465, 0.4943,  ..., 0.5116, 0.5539, 0.2465],
         [0.8586, 0.2493, 0.4540,  ..., 0.5116, 0.4466, 0.2493]],

        [[0.8586, 0.1717, 0.4713,  ..., 0.5814, 0.5380, 0.1717],
         [0.8586, 0.1745, 0.4310,  ..., 0.7054, 0.7487, 

         [0.8990, 0.0055, 0.6092,  ..., 0.6744, 0.6031, 0.0055]]])
tensor([[[0.8889, 0.5291, 0.3563,  ..., 0.5039, 0.4642, 0.5291],
         [0.8889, 0.5319, 0.3851,  ..., 0.3256, 0.4681, 0.5319],
         [0.8889, 0.5346, 0.5690,  ..., 0.2791, 0.2306, 0.5346],
         ...,
         [0.8990, 0.0028, 0.5115,  ..., 0.8372, 0.6095, 0.0028],
         [0.8990, 0.0055, 0.6092,  ..., 0.6744, 0.6031, 0.0055],
         [0.8990, 0.0083, 0.7184,  ..., 0.7364, 0.7549, 0.0083]],

        [[0.8889, 0.5319, 0.3851,  ..., 0.3256, 0.4681, 0.5319],
         [0.8889, 0.5346, 0.5690,  ..., 0.2791, 0.2306, 0.5346],
         [0.8889, 0.5374, 0.4310,  ..., 0.2946, 0.3719, 0.5374],
         ...,
         [0.8990, 0.0055, 0.6092,  ..., 0.6744, 0.6031, 0.0055],
         [0.8990, 0.0083, 0.7184,  ..., 0.7364, 0.7549, 0.0083],
         [0.8990, 0.0111, 0.5805,  ..., 0.7054, 0.7966, 0.0111]],

        [[0.8889, 0.5346, 0.5690,  ..., 0.2791, 0.2306, 0.5346],
         [0.8889, 0.5374, 0.4310,  ..., 0.2946, 0.3719, 

         [0.9394, 0.6011, 0.5172,  ..., 0.4806, 0.5052, 0.6011]]])
tensor([[[0.9394, 0.5235, 0.6609,  ..., 0.4729, 0.4205, 0.5235],
         [0.9394, 0.5263, 0.5345,  ..., 0.4651, 0.4492, 0.5263],
         [0.9394, 0.5291, 0.6724,  ..., 0.4961, 0.5115, 0.5291],
         ...,
         [0.9394, 0.5983, 0.5230,  ..., 0.2868, 0.3093, 0.5983],
         [0.9394, 0.6011, 0.5172,  ..., 0.4806, 0.5052, 0.6011],
         [0.9394, 0.6039, 0.6782,  ..., 0.3178, 0.3079, 0.6039]],

        [[0.9394, 0.5263, 0.5345,  ..., 0.4651, 0.4492, 0.5263],
         [0.9394, 0.5291, 0.6724,  ..., 0.4961, 0.5115, 0.5291],
         [0.9394, 0.5319, 0.5920,  ..., 0.4806, 0.5373, 0.5319],
         ...,
         [0.9394, 0.6011, 0.5172,  ..., 0.4806, 0.5052, 0.6011],
         [0.9394, 0.6039, 0.6782,  ..., 0.3178, 0.3079, 0.6039],
         [0.9394, 0.6066, 0.5690,  ..., 0.4031, 0.3788, 0.6066]],

        [[0.9394, 0.5291, 0.6724,  ..., 0.4961, 0.5115, 0.5291],
         [0.9394, 0.5319, 0.5920,  ..., 0.4806, 0.5373, 

         [0.9697, 0.3878, 0.5057,  ..., 0.8295, 0.6523, 0.3878]]])
tensor([[[0.9697, 0.3102, 0.4655,  ..., 0.5891, 0.7088, 0.3102],
         [0.9697, 0.3130, 0.6034,  ..., 0.5581, 0.7135, 0.3130],
         [0.9697, 0.3158, 0.3563,  ..., 0.5426, 0.5677, 0.3158],
         ...,
         [0.9697, 0.3850, 0.2299,  ..., 0.6434, 0.6117, 0.3850],
         [0.9697, 0.3878, 0.5057,  ..., 0.8295, 0.6523, 0.3878],
         [0.9697, 0.3906, 0.4310,  ..., 0.7829, 0.5003, 0.3906]],

        [[0.9697, 0.3130, 0.6034,  ..., 0.5581, 0.7135, 0.3130],
         [0.9697, 0.3158, 0.3563,  ..., 0.5426, 0.5677, 0.3158],
         [0.9697, 0.3186, 0.4828,  ..., 0.5349, 0.6371, 0.3186],
         ...,
         [0.9697, 0.3878, 0.5057,  ..., 0.8295, 0.6523, 0.3878],
         [0.9697, 0.3906, 0.4310,  ..., 0.7829, 0.5003, 0.3906],
         [0.9697, 0.3934, 0.4885,  ..., 0.6667, 0.4920, 0.3934]],

        [[0.9697, 0.3158, 0.3563,  ..., 0.5426, 0.5677, 0.3158],
         [0.9697, 0.3186, 0.4828,  ..., 0.5349, 0.6371, 

In [19]:
# this is just to configure model hyperparameters
# Input configurations
input_size = 20      # since one row has 20 features, we are reading one row at a time
sequence_length = 30 # since there are 30 rows 
num_layers = 2       # stack 2 RNN together

# Hyperparameter
hidden_size = 128 # i think this is the number of hidden nodes
num_classes = 2
epochs = 5
# batch_size = 200
learning_rate = 0.001

random_seed = 42

torch.manual_seed(random_seed) # to ensure reproducivility

In [21]:
# Let's instantiate a model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__() # this is for backward compatibility of Python 2, same as super().__init()
        self.hidden_size = hidden_size # this is to set attribute of the instance
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) # in this order, so (28, 128, 2)
                                                                                 # batch_first set number of batch as 1st dimension
        # x -> (batch_size, seq, input_size) [the shape needed for the tensor]
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) # initiate zeros tensor with (num_layers, batch_size, hidden_size)
        
        out, _ = self.rnn(x, h0)
        # output shape: batch_size, seq_length, hidden_size
        # out (N, 28, 128)
        out = out[:, -1, :]
        # out (N, 128)
        out = self.fc(out)
        return out
    
model = RNN(input_size, hidden_size, num_layers, num_classes)
print(model)

# let's set loss function and optimizer
criterion = nn.BCEWithLogitsLoss() # since this is a binary class problem
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# finally we can start to train
n_total_steps = len(train_loader)
print(n_total_steps)

for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):          
        # forward pass
        outputs = model(x)
        y = y.type_as(outputs)
        loss = criterion(outputs, y)
        
        # backward pass
        optimizer.zero_grad() # this is to clear the parameters, done before each backward pass
        loss.backward()
        optimizer.step()
        
        # print out the training performance
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():10.4f}')
            
# Let's evaluate our model
# Remember, we don't need to compute gradients as it is not required (and save some precious memory too!)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for x, y in test_loader:
        outputs = model(x)
        # max returns (value, index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += y.size(0)
        n_correct += (predicted == y).sum().item()
        
acc = 100.0 * n_correct / n_samples
print(f"Accuracy of the network on the 10000 test images: {acc:10.2f} %")

RNN(
  (rnn): RNN(20, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)
104
Epoch [1/5], Step [100/104], Loss:     0.1858
Epoch [2/5], Step [100/104], Loss:     0.1439
Epoch [3/5], Step [100/104], Loss:     0.1367
Epoch [4/5], Step [100/104], Loss:     0.1320
Epoch [5/5], Step [100/104], Loss:     0.1256
Accuracy of the network on the 10000 test images:     100.00 %


Well, our model has a great accuracy! But this does not mean that it is a very good model, probably is due to the imbalance class issue and that the model just predicted the major classes. We can check out other classification metrics like precision, recall, and F1-score to evaluate it. 

References:
1. https://jovian.ai/aakanksha-ns/shelter-outcome
2. https://stackoverflow.com/questions/50307707/convert-pandas-dataframe-to-pytorch-tensor
3. https://stackoverflow.com/questions/62208904/pytorch-custom-dataset-dataloader-returns-a-list-of-tensors-rather-than-tensor
